### Intro
고객의 신용 한도, 최근 5개월 사용 내역, 지급 등으로 부터 카드 default 가능성을 예측합니다.  
* 오래된 공개데이터라 변수별 상세 설명 부족  
* 데이터 값에 일부 오류 존재하지만 전처리 과정에서 일부분 정리  
* 전처리 데이터는 scikit-learn, keras 에서 바로 사용할 수 있는 형태로 구성  
    * 해당 라이브러리에는 기계학습, 딥러닝을 위한 학습 및 평가 모듈이 이미 구성
    * 해당 함수들의 호춤만으로 간단한 예측 모델링 가능  

### models
* SVM, 인공신경망(MLP)  
* 각 데이터 변수의 특징과 영향력에 대한 분석등은 생략
    * 해당 데이터 변수에 대한 분포등을 살펴보고, 해당 데이터의 삭제, 변형 등이 결과값을 미치는 영향을 살펴보기 권장  

In [1]:
import numpy as np 
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import keras
from keras import models
from keras import layers

/home/bwlee/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### 데이터 로드
y 값은 (0, 1) 사이의 값  
ys = [1, 0, 0, 1, ...]

In [3]:
import sys
import card_default as cd

# load data
data1 = cd.Data()
data1.load_data()
(x_train, y_train), (x_test, y_test) = data1.load_data()

#### SVM
scikit-learn 모듈을 사용해 SVM 적용
(딥러닝 라이브러리 실행시 시스템 상태에 대한 출력이 많아 편의상 결과값을 아래에서 출력)

In [4]:
estimator = SVC(kernel='rbf')
selector = estimator.fit(x_train,y_train)

y_predict_train = selector.predict(x_train)
y_predict_test = selector.predict(x_test)

/home/bwlee/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### MLP
y값을 다차원 카테고리 형식으로 바꾸어 주고, 학습 후 동일한 방식으로 비교  
[1, 0, ...] --> [[0,1], [1,0], ...]

In [5]:
from keras.utils import to_categorical
y_test_cat = to_categorical(y_test)
y_train_cat = to_categorical(y_train)

mlp = models.Sequential()
mlp.add(layers.Dense(50, activation='relu', input_shape=(30,)))
mlp.add(layers.Dense(2, activation='softmax'))
mlp.compile(optimizer='adagrad', 
            loss='categorical_crossentropy', metrics=['accuracy'])

mlp.fit(x_train, y_train_cat, epochs=50, batch_size = 100, verbose=0)
loss0, accuracy0 = mlp.evaluate(x_train, y_train_cat)
loss, accuracy = mlp.evaluate(x_test, y_test_cat)

7500/7500 [==============================] - 0s 15us/step


#### Results

In [6]:
print("======== SVM ========")
print("Train Accuracy Score:", accuracy_score(y_train, y_predict_train))
print("Test Accuracy Score:", accuracy_score(y_test, y_predict_test))

print("======== MLP ========")
print('train accuracy = ', accuracy0)
print('test accuracy = ', accuracy)

======== SVM ========
Train Accuracy Score: 0.7828888888888889
Test Accuracy Score: 0.7817333333333333
======== MLP ========
train accuracy =  0.8139555555555555
test accuracy =  0.8132000000635783
